## Table of contents
1. [Introduction](#Introduction)
2. [Quickstart](#Quickstart)
3. [Applying styles](#Applying-styles)
4. [Relative positioning](#Relative-positioning)

## Introduction

## Quickstart

In [1]:
import IPython.display
import tempfile
import os
import time

In [2]:
import momapy.sbgn.io
import momapy.sbgn.styling
import momapy.sbgn.utils
import momapy.rendering
import momapy.builder
import momapy.coloring
import momapy.styling

In [3]:
filenames = []
def display(map_, width=450, height=300):
    if momapy.builder.isinstance_or_builder(map_, (momapy.core.Map, momapy.core.LayoutElement)):
        f, filename = tempfile.mkstemp(dir=".", suffix=".svg")
        if momapy.builder.isinstance_or_builder(map_, (momapy.core.LayoutElement)):
            if isinstance(map_, momapy.core.LayoutElement):
                map_ = momapy.builder.builder_from_object(map_)
            map_layout = momapy.core.MapLayoutBuilder()
            map_layout.add_element(map_)
            map_layout.width = width
            map_layout.height = height
            map_layout.position = momapy.geometry.PointBuilder(width / 2, height / 2)
            map_ = momapy.sbgn.pd.SBGNPDMapBuilder(layout=map_layout)
        momapy.rendering.render_map(map_, filename, renderer="svg-native")
    else:
        filename = map_
    print(os.path.split(filename)[-1])
    s = "<img src='{}' width={}>".format(os.path.split(filename)[-1], width)
    IPython.display.display(IPython.display.HTML(s))
    filenames.append(filename)
    
def show_room(cls, type_="anchor"):
    WIDTH = 450
    HEIGHT = 300
    POSITION = momapy.geometry.PointBuilder(WIDTH / 2, HEIGHT / 2)
    SCALE = 5
    FONT_SIZE = 12
    DISTANCE = 6
    CROSS_SIZE = 12
    ANCHORS = {
        "north_west": "south_east",
        "north": "south",
        "north_east": "south_west",
        "east": "west",
        "south_east": "north_west",
        "south": "north",
        "south_west": "north_east",
        "west": "east",
        "center": "center",
        "label_center": "label_center"
    }
    ANGLE_STEP = 15
    AUXILLIARY_UNIT_WIDTH = 18
    AUXILLIARY_UNIT_HEIGHT = 9
    if not issubclass(cls, momapy.builder.Builder):
        cls = momapy.builder.get_or_make_builder_cls(cls)
    m = cls()
    m.position=POSITION
    m.width = m.width * SCALE
    m.height = m.height * SCALE
    for attr in ["offset", "left_connector_length", "right_connector_length"]: 
        if hasattr(m, attr):
            setattr(m, attr, getattr(m, attr) * SCALE)
    if type_ == "angle":
        StateVariableLayoutBuilder = momapy.builder.get_or_make_builder_cls(momapy.sbgn.pd.StateVariableLayout)
        UnitOfInformationLayoutBuilder = momapy.builder.get_or_make_builder_cls(momapy.sbgn.pd.UnitOfInformationLayout)
        s1 = StateVariableLayoutBuilder(
            width=AUXILLIARY_UNIT_WIDTH * SCALE,
            height=AUXILLIARY_UNIT_HEIGHT * SCALE,
            position=m.self_angle(130)
        )
        m.add_element(s1)
        s2 = StateVariableLayoutBuilder(
            width=AUXILLIARY_UNIT_WIDTH * SCALE,
            height=AUXILLIARY_UNIT_HEIGHT * SCALE,
            position=m.self_angle(50)
        )
        m.add_element(s2)
        u1 = UnitOfInformationLayoutBuilder(
            width=AUXILLIARY_UNIT_WIDTH * SCALE,
            height=AUXILLIARY_UNIT_HEIGHT * SCALE,
            position=m.south()
        )
        m.add_element(u1)
    CrossPointBuilder = momapy.builder.get_or_make_builder_cls(momapy.shapes.CrossPoint)
    if type_ == "anchor":
        l = ANCHORS.keys()
    elif type_ == "angle" or type_ == "self_angle":
        l = range(0, 360, ANGLE_STEP)
    for i, elem in enumerate(l):
        if type_ == "anchor":
            position = getattr(m, elem)()
            text = elem
        elif type_ == "angle":
            position = m.angle(elem)
            text = str(elem)
        elif type_ == "self_angle":
            position = m.self_angle(elem)
            text = str(elem)
        cross = CrossPointBuilder(
            width = CROSS_SIZE,
            height = CROSS_SIZE,
            position = position,
            stroke_width = 1.5,
            stroke = momapy.coloring.colors.red,
            label = momapy.core.TextLayoutBuilder(
                text=text,
                font_family = "Arial",
                font_size = FONT_SIZE,
            ),
        )
        if type_ == "anchor":
            if elem == "label_center":
                func_name = "set_below_left_of"
                attach = "north_east"
            elif elem == "center":
                func_name = "set_above_right_of"
                attach = "south_west"
            else:
                func_name = "set_{}_of".format(elem.replace("north", "above").replace("east", "right").replace("south", "below").replace("west", "left"))
                attach = ANCHORS[elem]
        elif type_ == "self_angle" or type_ == "angle":
            if i % 2 == 0:
                func_name = "set_above_right_of"
                attach = "south_west"
            else:
                func_name = "set_below_right_of"
                attach = "north_west"
        getattr(momapy.positioning, func_name)(cross.label, cross, DISTANCE, anchor=attach)
        m.add_element(cross)
    map_layout = momapy.core.MapLayoutBuilder(
        layout_elements=momapy.core.TupleBuilder([m]),
        position=POSITION,
        width=WIDTH,
        height=HEIGHT
    )
    map_ = momapy.sbgn.pd.SBGNPDMapBuilder(layout=map_layout)
    display(map_)
    
def cleanup():
    time.sleep(2)
    for filename in filenames:
        os.remove(filename)

In [4]:
m1 = momapy.sbgn.io.read_file("phospho1.sbgn")
display(m1, 400)

tmpaw6mrfsv.svg


In [5]:
m2 = momapy.sbgn.io.read_file("phospho2.sbgn")
display(m2, 300)

tmp09ccef5b.svg


In [6]:
m1 == m2

False

In [7]:
m1.layout == m2.layout

False

In [8]:
m1.model == m2.model

True

In [9]:
try:
    m1.layout.stroke = momapy.coloring.colors.red
except Exception as e:
    print(e)

cannot assign to field 'stroke'


In [10]:
m1_builder = momapy.builder.builder_from_object(m1)

In [11]:
m1_builder.layout.stroke = momapy.coloring.colors.red
m1_builder.layout.stroke_width = 3
display(m1_builder, 400)

tmpg4ww9fu0.svg


In [12]:
m1_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)

In [13]:
m1_builder.layout.stroke = momapy.coloring.colors.red
m1_builder.layout.stroke_width = 3
display(m1_builder, 400)

tmpr6jp__0i.svg


In [14]:
m1 = momapy.sbgn.io.read_file("phospho1.sbgn")
m1_builder = momapy.builder.builder_from_object(m1)
m1 == m1_builder.build()

True

### So what is the purpose of the non builder object?

In [15]:
m1a_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m1b_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m1a_builder == m1b_builder

False

In [16]:
m1a = m1a_builder.build()
m1b = m1b_builder.build()
m1a == m1b

True

In [17]:
m1_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m1_builder.layout.stroke_width = 3
m1_builder.layout.stroke = momapy.coloring.colors.red
m1 = m1_builder.build()

In [18]:
s = set([])
for i in range(256):
    for j in range(0):
            m1_builder.layout.stroke = momapy.coloring.rgba(i, j, 0, 1.0)
            s.add(m1_builder.build())
l = list(s)

In [19]:
%%time
m1 in s

CPU times: user 95 µs, sys: 32 µs, total: 127 µs
Wall time: 131 µs


False

In [20]:
%%time
m1 in l

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


False

### Submap

In [21]:
m1_builder = momapy.sbgn.io.read_file("phospho1comp.sbgn", return_builder=True)
m1 = m1_builder.build()
display(m1)

tmpz5fkq7zp.svg


In [22]:
m2_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
m2_builder.layout.position = m1_builder.layout.position
m2_builder.layout.width = m1_builder.layout.width
m2_builder.layout.height = m1_builder.layout.height
m2 = m2_builder.build()
display(m2)

tmp2huc3z2x.svg


In [23]:
m1.is_submap(m2)

False

In [24]:
m1.layout.is_sublayout(m2.layout)

True

In [25]:
m1.model.is_submodel(m2.model)

False

## Applying styles

In [26]:
m1_builder = momapy.sbgn.io.read_file("phospho1.sbgn", return_builder=True)
momapy.sbgn.utils.set_layout_to_fit_content(m1_builder, xsep=10, ysep=10)

In [27]:
display(m1_builder)

tmpwrxfhr0g.svg


In [28]:
momapy.styling.apply_style_sheet(m1_builder.layout, momapy.sbgn.styling.default_colorscheme)
display(m1_builder)

tmpfzr0w06w.svg


In [29]:
momapy.styling.apply_style_sheet(m1_builder.layout, momapy.sbgn.styling.shadows)
display(m1_builder)

tmpgsj5k2vg.svg


In [30]:
momapy.styling.apply_style_sheet(m1_builder.layout, momapy.sbgn.styling.newt)
display(m1_builder)

tmp7bved5ws.svg


In [31]:
momapy.sbgn.utils.set_auxilliary_units_to_borders(m1_builder)
momapy.sbgn.utils.set_arcs_to_borders(m1_builder)
display(m1_builder)

tmp6xg67zrs.svg


In [32]:
momapy.styling.apply_style_sheet(m1_builder.layout, momapy.sbgn.styling.vanted)
display(m1_builder)

tmpc30944ts.svg


In [33]:
momapy.sbgn.utils.set_auxilliary_units_to_borders(m1_builder)
momapy.sbgn.utils.set_arcs_to_borders(m1_builder)
display(m1_builder)

tmp4uq6rti5.svg


In [34]:
s = """
    MacromoleculeLayout {
        stroke_width: 3;
        fill: red;
    }
    
    StateVariableLayout > TextLayout {
        font_color: green;
    }
    """
my_style_sheet = momapy.styling.read_string(s)
momapy.styling.apply_style_sheet(m1_builder.layout, my_style_sheet)
display(m1_builder)

tmp2t20pl2m.svg


In [35]:
s = s.replace("red", "blue")
with open("my_style_sheet.css", "w") as f:
    f.write(s)
my_style_sheet = momapy.styling.read_file("my_style_sheet.css")
momapy.styling.apply_style_sheet(m1_builder.layout, my_style_sheet)
display(m1_builder)

tmp35s48dxq.svg


## Relative positioning

In [36]:
MacromoleculeLayoutBuilder = momapy.builder.get_or_make_builder_cls(momapy.sbgn.pd.MacromoleculeLayout)
m = MacromoleculeLayoutBuilder(
        position=momapy.geometry.PointBuilder(225, 150),
        width=300,
        height=150,
        stroke_width=1,
        rounded_corners=15,
        stroke=momapy.coloring.colors.black,
)

In [37]:
m.north_west()

Point(x=80.99999999999999, y=78.00000000000001)

In [38]:
CrossPointBuilder = momapy.builder.get_or_make_builder_cls(momapy.shapes.CrossPoint)
cross_point = CrossPointBuilder(
    width=12.0,
    height=12.0,
    stroke_width=1.5,
    stroke=momapy.coloring.colors.red
)
cross_point.position = m.north_west()
m.add_element(cross_point)
display(m)

tmpbbh20al1.svg


All nodes have at least the following anchor points:
* north_west
* north
* north_east
* east
* south_east
* south
* south_west
* west
* center
* label_center

In [39]:
show_room(momapy.sbgn.pd.MacromoleculeLayout)

tmpu_mdtgvp.svg


In [40]:
show_room(momapy.sbgn.pd.GenericProcessLayout)

tmp22hmhxh2.svg


In [41]:
show_room(momapy.sbgn.pd.ComplexMultimerLayout)

tmpmtiheama.svg


In [42]:
m.self_angle(120)

Point(x=181.69872981077808, y=74.99999999999999)

In [43]:
cross_point.position = m.self_angle(120)
display(m)

tmpamfgivji.svg


In [44]:
show_room(momapy.sbgn.pd.ComplexMultimerLayout, "self_angle")

tmp6flmjqga.svg


In [45]:
m.layout_elements.clear()
StateVariableLayoutBuilder = momapy.builder.get_or_make_builder_cls(momapy.sbgn.pd.StateVariableLayout)

In [46]:
s = StateVariableLayoutBuilder(
    width=90,
    height=45
)
s.position = m.self_angle(120)
m.add_element(s)
cross_point.position = m.self_angle(120)
m.add_element(cross_point)
display(m)

tmp9iisbm4w.svg


In [47]:
cross_point.position = m.angle(120)
display(m)

tmpvymw15ub.svg


In [48]:
show_room(momapy.sbgn.pd.MacromoleculeLayout, "angle")

tmpjd8784oh.svg
